In [1]:
pip install spacy

Note: you may need to restart the kernel to use updated packages.


In [2]:
!python -m spacy download en_core_web_sm


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.1/12.8 MB 744.7 kB/s eta 0:00:18
     --------------------------------------- 0.1/12.8 MB 744.7 kB/s eta 0:00:18
      -------------------------------------- 0.2/12.8 MB 888.4 kB/s eta 0:00:15
      -------------------------------------- 0.2/12.8 MB 888.4 kB/s eta 0:00:15
      -------------------------------------- 0.2/12.8 MB 888.4 kB/s eta 0:00:15
      -------------------------------------- 0.3/12.8 MB 967.8 kB/s eta 0:00:13
      -------------------------------------- 0.3/12.8 MB 967.8 kB/s eta 0:00:13
     - ------------------------------------- 0.4/12.8 MB 994.9 kB/s eta 0:00:13


In [3]:
import re
import spacy
from spacy.matcher import Matcher
import pymssql
import pandas as pd

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Initialize matchers for weapons and terrorist organizations
weapon_matcher = Matcher(nlp.vocab)
terrorist_org_matcher = Matcher(nlp.vocab)
city_matcher = Matcher(nlp.vocab)
state_matcher = Matcher(nlp.vocab)

# Define custom patterns for weapons
weapon_patterns = [
    [{"LOWER": {"IN": ["pistol", "revolver", "rifle", "bulldozer", "stones", "shotgun", "uzi", "mp5", "knife", "katana", "sabre", "axe", "tomahawk", "dagger", "stiletto", "dirk", "grenade", "bomb", "club", "bat", "spear", "halberd", "bow", "crossbow", "taser", "stun", "gun", "pepper", "spray", "rubber", "bullet"]}}],
    [{"LOWER": "assault"}, {"LOWER": "rifle"}],
    [{"LOWER": "sniper"}, {"LOWER": "rifle"}],
    [{"LOWER": "submachine"}, {"LOWER": "gun"}],
    [{"LOWER": "machine"}, {"LOWER": "gun"}],
    [{"LOWER": "combat"}, {"LOWER": "knife"}],
    [{"LOWER": "bowie"}, {"LOWER": "knife"}],
    [{"LOWER": "suicide"}, {"LOWER": "bombing"}],
    [{"LOWER": "landmine"}],
    [{"LOWER": "naval"}, {"LOWER": "mine"}],
    [{"LOWER": "improvised"}, {"LOWER": "explosive"}, {"LOWER": "device"}],
    [{"LOWER": "howitzer"}],
    [{"LOWER": "mortar"}],
    [{"LOWER": "main"}, {"LOWER": "battle"}, {"LOWER": "tank"}],
]
weapon_matcher.add("WEAPON", weapon_patterns)

# Define custom patterns for terrorist organizations
terrorist_org_patterns = [
   [{"LOWER": {"IN": ["al-qaeda", "isis", "taliban", "boko haram", "hamas", "hezbollah", "lashkar-e-taiba", "farc", "al-shabaab", "pkk", "ira", "abu sayyaf"]}}],
   [{"LOWER": {"IN": ["al", "qaeda"]}}, {"LOWER": "affiliated"}],
   [{"LOWER": {"IN": ["islamic", "state"]}}, {"LOWER": "of"}, {"LOWER": {"IN": ["iraq", "syria"]}}],
   [{"LOWER": "islamic"}, {"LOWER": "state"}],
   [{"LOWER": "revolutionary"}, {"LOWER": "armed"}, {"LOWER": "forces"}, {"LOWER": "of"}, {"LOWER": "colombia"}],
   [{"LOWER": "kurdistan"}, {"LOWER": "workers"}, {"LOWER": "party"}],
]

terrorist_org_matcher.add("TERRORIST_ORG", terrorist_org_patterns)

cities_patterns = [
    [{"LOWER": {"IN": ["abohar", "adilabad", "agartala", "agra", "ahmadnagar", "ahmedabad", "aizawl", "ajmer", "akola", "alappuzha", "aligarh", 
                       "alipurduar", "allahabad", "alwar", "ambala", "amaravati", "amritsar", "asansol", "aurangabad", "bakshpur", "bamanpuri", "baramula", "barddhaman", "bareilly", "belgaum", "bellary", 
                       "bengaluru", "bhagalpur", "bharatpur","bharauri", "bhatpara", "bhavnagar", "bhilai", "bhilwara", "bhiwandi", "bhiwani", "bhopal", "bhubaneshwar", "bhuj", "bhusaval", "bidar", "bijapur", 
                       "bikaner", "bilaspur", "brahmapur", "budaun", "bulandshahr", "calicut", "chanda", "chandigarh", "chennai", "chikka", "mandya", "chirala", "coimbatore", 
                       "cuddalore", "cuttack", "daman", "davangere", "dehradun", "delhi", "dhanbad", "dibrugarh", "dindigul", "dispur", "diu", "faridabad", "firozabad", 
                       "fyzabad", "gangtok", "gaya", "gandhinagar", "ghaziabad", "gopalpur", "gulbarga", "guntur", "gurugram", "guwahati", "gwalior", "haldia", "haora", 
                       "hapur", "haripur", "hata", "hindupur", "hisar", "hospet", "hubli", "hyderabad", "imphal", "indore", "itanagar", "jabalpur", "jaipur", "jammu","jammu and kashmir" 
                       "jamshedpur", "jhansi", "jodhpur", "jorhat", "kagaznagar", "kakinada", "kalyan", "karimnagar", "karnal", "karur", "kavaratti", "khammam", "khanapur", 
                       "kochi", "kohima", "kolar", "kolhapur", "kolkata", "kollam", "kota", "krishnanagar", "krishnapuram", "kumbakonam", "kurnool", "latur", "lucknow", 
                       "ludhiana", "machilipatnam", "madurai", "mahabubnagar", "malegaon","mangalore", "mathura", "meerut", "mirzapur", "moradabad", "mumbai", 
                       "muzaffarnagar", "muzaffarpur", "mysore", "nagercoil", "nalgonda", "nanded", "nandyal", "nasik", "navsari", "nellore"," new delhi", "nizamabad", 
                       "ongole", "pali", "panaji", "panchkula", "panipat", "parbhani", "pathankot", "patiala", "patna", "pilibhit", "porbandar", "port", "blair", "proddatur", 
                       "puducherry", "pune", "puri", "purnea", "raichur", "raipur", "rajahmundry", "rajapalaiyam", "rajkot", "ramagundam", "rampura", "ranchi", "ratlam", 
                       "raurkela", "rohtak", "saharanpur", "saidapur", "salem", "samlaipadar", "sangli", "saugor", "shahbazpur", "shiliguri", "shillong", "shimla", "shimoga", 
                       "sikar", "silchar", "silvassa", "Sirsa", "sonipat", "srinagar", "surat", "tezpur", "thanjavur", "tharati", "etawah", "thiruvananthapuram", 
                       "tiruchchirappalli", "tirunelveli", "tirupati", "tiruvannamalai", "tonk", "tuticorin", "udaipur", "ujjain", "vadodara", "valparai", "varanasi",
                       "vellore", "vishakhapatnam", "vizianagaram", "warangal", "jorapokhar", "brajrajnagar", "talcher"]}}]
]

city_matcher.add("City", cities_patterns)

states_patterns = [
    [{"LOWER": {"IN": [
        "andhra pradesh", "arunachal pradesh", "assam", "bihar", "chhattisgarh", "goa", "gujarat", 
        "haryana", "himachal pradesh", "jharkhand", "karnataka", "kerala", "madhya pradesh", 
        "maharashtra", "manipur", "meghalaya", "mizoram", "nagaland", "odisha", "punjab", 
        "rajasthan", "sikkim", "tamil nadu", "telangana", "tripura", "uttar pradesh", 
        "uttarakhand", "west bengal", "andaman and nicobar islands", "chandigarh", 
        "dadra and nagar haveli and daman and diu", "lakshadweep", "puducherry"
    ]}}]
]

state_matcher.add("State",states_patterns)

# Function to classify information type using keywords
def classify_information_type(text):
    event_keywords = ["event", "festival", "celebration", "concert", "parade", "conference", "meeting"]
    incident_keywords = ["attack", "bombing", "shooting", "incident", "assault", "murder", "crime"]
    alert_keywords = ["alert", "warning", "notice", "advisory", "caution", "bulletin"]

    text_lower = text.lower()

    if any(keyword in text_lower for keyword in event_keywords):
        return "Event"
    elif any(keyword in text_lower for keyword in incident_keywords):
        return "Incident"
    elif any(keyword in text_lower for keyword in alert_keywords):
        return "Alert"
    else:
        return "Unknown"

# Function to extract date of incident
from spacy.tokens import Span

# Enhanced function to extract date of incident
def extract_date_of_incident(doc):
    
    incident_keywords = {
    "attack", "incident", "bombing", "shooting", "assault", "murder", 
    "explosion", "fire", "hostage", "hijacking", "massacre", "raid", 
    "terrorism", "kidnapping", "clash", "confrontation", "ambush", 
    "violence", "war", "battle", "conflict", "skirmish", "assassination", 
    "detonation", "destruction", "gunfire", "gunfight", "hostilities", 
    "insurgency", "killings", "lynching", "mass murder", "militancy", 
    "offensive", "shootout", "slaughter", "suicide attack", "uprising", 
    "vehicular attack", "vandalism", "civil unrest", "riot", "revolt", 
    "insurrection", "aerial attack", "airstrike", "arson", "bomb blast", 
    "border clash", "car bomb", "civil disturbance", "coup", "crime spree", 
    "death toll", "demonstration", "extremism", "fatality", "genocide", 
    "gun battle", "homicide", "imprisonment", "intifada", "military action", 
    "murder-suicide", "political violence", "protest", "rebellion", "revolution", 
    "sabotage", "sectarian violence", "siege", "street fight", "subversion", 
    "suppression", "terrorist act", "torture", "turmoil", "urban warfare", 
    "weapon discharge", "weaponized attack", "ethnic cleansing", 
    "forced displacement", "martial law incident", "accusation", "agreement", 
    "announcement", "bill", "campaign", "candidate", "coalition", "conference", 
    "debate", "decision", "declaration", "diplomacy", "dispute", "election", 
    "endorsement", "government", "hearing", "investigation", "judgment", "law", 
    "legislation", "meeting", "negotiation", "parliament", "policy", "president", 
    "prime minister", "proposal", "reform", "resolution", "resignation", 
    "sanction", "scandal", "speech", "statement", "summit", "treaty", "vote"
}
    
    context_window = 5  # Number of tokens before and after the date to consider as context
    
    def normalize_date(date_text):
        # Implement normalization logic (e.g., convert "today" or "yesterday" to actual dates)
        return date_text.strip().rstrip(" night").rstrip(" evening")
    
    for ent in doc.ents:
        if ent.label_ == "DATE":
            # Extract tokens around the date for context
            start_idx = max(ent.start - context_window, 0)
            end_idx = min(ent.end + context_window, len(doc))
            context_tokens = doc[start_idx:end_idx]
            
            # Check if any of the context tokens are incident-related
            for token in context_tokens:
                if token.text.lower() in incident_keywords:
                    return normalize_date(ent.text)
            
            # If no reinforcing token found, check the sentence
            if isinstance(ent, Span):
                sentence = ent.sent
                if any(token.lower_ in incident_keywords for token in sentence):
                    return normalize_date(ent.text)
    
    # If no relevant date is found, return "Unknown"
    return "Unknown"

# Function to extract segments from each line
def extract_segments(line):
    try:
        doc = nlp(line)  # Process line using spaCy
        entities = {
            "Id": None,
            "PostContent": line.strip(),
            "InformationType": [],
            "Organizations":[],
            "State": [],
            "District": [],
            "City": [],
            "PersonInvolved": [],
            "Weapons": [],
            "DateOfIncident": [],
            "EventType": [],
            "TerroristOrganizations": [],
            "Norp": [],
            "DatesMentioned": [],
            "Gpe": [],
            "Loc": [],
            "Cardinal": [],
            "Norp": [],
            "Money": [],
            "Fac": [],
            "Time": [],     
            
        }  # Initialize dictionary for entities
        
        # Initialize sets to store unique entities detected in the line
        unique_weapons = set()
        unique_terrorist_org = set()
        unique_persons = set()
        unique_dates = set()
        unique_gpe = set()
        unique_loc = set()
        unique_cardinal = set()
        unique_norp = set()
        unique_money = set()
        unique_time = set()
        unique_event = set()
        unique_quantity = set()
        unique_org = set()
        unique_state = set()
        unique_city = set()
        unique_district = set()
        unique_fac = set()

        # Apply the weapon matcher to the doc
        weapon_matches = weapon_matcher(doc)
        for match_id, start, end in weapon_matches:
            span = doc[start:end]
            unique_weapons.add(span.text)

        # Apply the terrorist organization matcher to the doc
        terrorist_org_matches = terrorist_org_matcher(doc)
        for match_id, start, end in terrorist_org_matches:
            span = doc[start:end]
            unique_terrorist_org.add(span.text)

        # Apply the city matcher to the doc
        city_matches = city_matcher(doc)
        for match_id, start, end in city_matches:
            span = doc[start:end]
            unique_city.add(span.text)
            
        # Apply the state matcher to the doc
        state_matches = state_matcher(doc)
        for match_id, start, end in state_matches:
            span = doc[start:end]
            unique_state.add(span.text)

        # Extract entities from the doc
        for ent in doc.ents:
            if ent.label_ == "PERSON":
                if ent.text not in unique_gpe:
                    unique_persons.add(ent.text)
            elif ent.label_ == "DATE":
                unique_dates.add(ent.text)
            elif ent.label_ == "GPE":
                unique_gpe.add(ent.text)
                gpe_text = ent.text.lower()
                if gpe_text not in [city.lower() for city in entities["City"]] and gpe_text not in [state.lower() for state in entities["State"]]:
                    unique_district.add(ent.text)
            elif ent.label_ == "LOC":
                unique_loc.add(ent.text)
            elif ent.label_ == "CARDINAL":
                unique_cardinal.add(ent.text)
            elif ent.label_ == "NORP":
                unique_norp.add(ent.text)
            elif ent.label_ == "MONEY":
                unique_money.add(ent.text)
            elif ent.label_ == "TIME":
                unique_time.add(ent.text)
            elif ent.label_ == "ORG":
                unique_org.add(ent.text)
            elif ent.label_ == "EVENT":
                unique_event.add(ent.text)
            elif ent.label_ == "FAC":
                unique_fac.add(ent.text)

        # Add the extracted entities to the dictionary
        entities["Weapons"] = list(unique_weapons)
        entities["TerroristOrganizations"] = list(unique_terrorist_org)
        entities["PersonInvolved"] = list(unique_persons)
        entities["DatesMentioned"] = list(unique_dates)
        entities["Gpe"] = list(unique_gpe)
        entities["Loc"] = list(unique_loc)
        entities["Cardinal"] = list(unique_cardinal)
        entities["Norp"] = list(unique_norp)
        entities["Money"] = list(unique_money)
        entities["Time"] = list(unique_time)
        entities["Organizations"] = list(unique_org)
        entities["EventType"]=list(unique_event)
        entities["City"]=list(unique_city)
        entities["State"]=list(unique_state)
        entities["District"] =list(unique_district)
        entities["Fac"]=list(unique_fac)
        
        # Classify information type and extract date of incident
        entities["InformationType"] = classify_information_type(line)
        entities["DateOfIncident"] = extract_date_of_incident(doc)

        return entities
    except Exception as e:
        print(f"Error processing line: {line}\n{str(e)}")
        return None  # Return None in case of an error

# Connect to SQL Server
conn = None
cursor = None

try:
    # Define connection parameters
    server = 'xx'
    database = 'PineData'
    username = 'dbadmin'
    password = 'VmaxDbAdmin@2024'

    # Establish connection to SQL Server
    conn = pymssql.connect(server, username, password, database)
    
    cursor = conn.cursor()
    sql = "SELECT Id, PostContent FROM [dbo].[NewsArticles]"
    cursor.execute(sql)
    rows = cursor.fetchall()

    # Initialize a list to store extracted segments
    data = []

    # Iterate over each row and extract segments
    for row in rows:
        Id = row[0]
        line = row[1] if isinstance(row, tuple) else row
        line = str(line)  # Ensure line is a string
        extracted_entities = extract_segments(line)
        if extracted_entities:
            extracted_entities["Id"] = Id
            data.append(extracted_entities)

    # Create a DataFrame from extracted segments
    output_df = pd.DataFrame(data)

    # Convert the lists of entities to comma-separated strings
    for col in output_df.columns:
        if isinstance(output_df[col][0], list):
            output_df[col] = output_df[col].apply(lambda x: ', '.join(x))

    # Display the DataFrame
    print(output_df)
    
    #save to excel file
    output_excel_file = 'outputsql_entitiesfinal.xlsx'
    output_df.to_excel(output_excel_file, index=False)
    
    try:
        # Update each row in the SQL table with corresponding extracted data
        for index, row in output_df.iterrows():
            sql_update = '''
            UPDATE [dbo].[NewsArticles]
            SET InformationType = %s,
                Organizations = %s,
                State = %s,
                District = %s,
                City = %s,
                PersonInvolved = %s,
                Weapons = %s,
                DateOfIncident = %s,
                EventType = %s,
                TerroristOrganizations = %s,
                Norp = %s,
                DatesMentioned = %s,
                Gpe = %s,
                Loc = %s,
                Cardinal = %s,
                Money = %s,
                Fac = %s,
                Time = %s
            WHERE Id = %s
            '''
            params = (
                row['InformationType'],
                row['Organizations'],
                row['State'],
                row['District'],
                row['City'],
                row['PersonInvolved'],
                row['Weapons'],
                row['DateOfIncident'],
                row['EventType'],
                row['TerroristOrganizations'],
                row['Norp'],
                row['DatesMentioned'],
                row['Gpe'],
                row['Loc'],
                row['Cardinal'],
                row['Money'],
                row['Fac'],
                row['Time'],
                row['Id']
            )
            
            cursor.execute(sql_update, params)
            conn.commit()

        print("Data updated successfully in SQL database.")
    
    except Exception as e:
        print(f"Error updating data: {str(e)}")
        conn.rollback()
    
finally:
    # Close cursor and connection
    if cursor:
        cursor.close()
    if conn:
        conn.close()


    Id                                        PostContent InformationType  \
0    1  The Financial Intelligence Unit has slapped a ...           Event   
1    6  Ottawa: Canada on Wednesday (local time) liste...           Event   
2    7  The Gulf region has become an increasingly imp...           Event   
3    9  KARACHI: Pakistan has called on the United Nat...           Event   
4   10  As India flexes its newly acquired muscles for...           Event   
5    2  The Financial Intelligence Unit (FIU) of India...           Event   
6    5  India's Consulate General in Toronto reaffirms...        Incident   
7   11  The Union government is considering introducin...           Event   
8    3  New Delhi, June 20: Leading blockchain and cry...           Event   
9    4  Financial Intelligence Unit penalised the comp...           Event   
10   8  Ottawa on Wednesday listed Iran's Revolutionar...           Event   
11  12  MUMBAI: Nawal Bajaj, a 1995-batch IPS officer,...         Unknown   